In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
data["transcription_text"][2]

'43M\r\nI Mrs E Montgomery solemnly Swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\nSgd in Montgomery\r\n\r\nSworn to and subscribed before me at Charleston SC this 31 day of Oct 1865.\r\nLevi Stuber\r\nMajor 47th Pa. Vet. Vols. and Provost Marshal.\r\n\r\n[[left margin]] Duplicate [[/left margin]]\r\n\r\nNo.1984\r\nUnited States of America.\r\nI Mrs E. Montgomery of the [[strikethrough]] County [[/strikethrough]]mnly Swear in the presence of Almighty God, that I will henceforth faithfully support and defend the constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves

In [4]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [5]:
txte= re.sub("([\(\[]).*?([\)\]])"," ",data["transcription_text"][10])
txte

' ]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics  ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxt

# Preprocessing

In [6]:
text=data["transcription_text"]

In [7]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [8]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [10]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:02<00:00,  8.52it/s]


In [11]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 2577.54it/s]


In [12]:
text[1]

'I E C Whaley solemnly swear I carefully read Amnesty Proclamation issued Andrew Johnson President of United State of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made except E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben Maj 47th PM United States of America E C Whaley of city county Charleston Sate of South Ca solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of SlavesSO HELP ME GOD E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben May 47 PM The fair complexion light hair gray eye 5 foot 10 inch high 39 year profession planter Copy Levi Steuben Maj 47th Pro Marshal City of Charleston'

# spacy

In [13]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])
        
NER(text)
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])

df1=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()

,entities,start_char,end_char,label,explain,frequency
0,Anne H Elliott,11,25,PERSON,"People, including fictional",
1,Amnesty,54,61,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,82,96,PERSON,"People, including fictional",
3,United States of America,110,134,GPE,"Countries, cities, states",
4,May 29th 1865,135,148,DATE,Absolute or relative dates or periods,


In [15]:
Person=[]
ORG=[]
DATE=[]
GPE=[]

for i in range(df1.shape[0]):
    if df1.label.loc[i]=="PERSON":
        Person.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="ORG" :
        ORG.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="DATE" :
        DATE.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="GPE" :
        GPE.append(df1.entities.loc[i])
GPE=np.array(GPE)
ORG=np.array(ORG)
Person=np.array(Person)
DATE=np.array(DATE)

In [16]:
DATE

array(['May 29th 1865', '27th day', '1865', '27th day', '1865', '63 year',
       'May 29th 1865', '28 day', '28 day', 'May 47 PM', '39 year',
       'May 29th 1865', '31 day', '1865', 'No1984', '31 day', '1865',
       '31st day', 'August 24 1865', 'May 29th 1865',
       '3 day of August 1865', 'May 4', '3 day', '1865', 'May 29th 1865',
       '16th day', '1474', '16th day', 'May 29th 1865',
       '16 day of October 1865', '16 day of October 1865', 'July 22',
       'July 22', '1867', 'May 29th 1865', '6 day', '6 day',
       '29th day of September 1865', 'Age 24 year', 'Sept 29th 1865',
       '29th day', 'May 29th 1865', '7th day of September',
       '7th day of September', '20th day of August 1865',
       '20th day of August 1865', 'August 29th 1865',
       '22nd day of August 1865', 'Miss 31st day', '1865 1865', '31 day',
       '1865'], dtype='<U26')

In [17]:
Person

array(['Anne H Elliott', 'Andrew Johnson', 'Hazel', 'Whaley',
       'Andrew Johnson', 'Copy Levi Steuben', 'Hu Kennedy',
       'Amnesty Oath Administered At', 'Hiserodt', 'William Whaley Sworn',
       'William Whaley Sworn', 'Brown', 'Charleston Sc',
       'JF Steinmeyer Sworn', 'Charleston Sc', 'Gry', 'Joseph Edings',
       'Thomas Andrew Oath of', 'Jacob Schroder Sworn', 'Walhalla',
       'Walhalla', 'Bray Lieut', 'Laborer', 'Characteristics',
       'Amnesty Oath', 'Woodville Wilkinson', 'Frank Conner',
       'Triplicate AMNESTY', 'Hiram Smith', 'Hiram Smith',
       'Pass Christian', 'Captn', 'Amnesty Oath', 'Hiram Smith',
       'Captn Commanding', 'Robert A',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Gibbs Henderson', 'City Miss', 'Amnesty Oath', 'Frank E Miller'],
      dtype='<U58')

# Hugging face  : Flair

**ner-english-ontonotes-large** : the best one on this type of text

In [18]:
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-

In [19]:
from flair.data import Sentence
from flair.models import SequenceTagger
T2=[]
D={}

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

for i in range(20):
        sentencess =Sentence(text[i])
        tagger.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T2.append(entity)
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                         D[entity.text]= entity.get_label("ner").value

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-03 14:31:02,449 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-03 14:31:22,600 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [20]:
len(D)

104

In [21]:
ent=[]
label=[]
for i in D :
    ent.append(i)
    label.append(D[i])

In [22]:
final1=pd.DataFrame({"entities":ent, "label":label})

In [23]:
P=[]
OR=[]
GP=[]
Date=[]
LOC=[]
for i in range(len(D)):
    if final1.label.loc[i]=="PERSON":
        P.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="ORG":
        OR.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="GPE":
        GP.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="DATE":
        Date.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="LOC":
        LOC.append(final1.entities.loc[i])
        

In [24]:
len(Date)

43

In [25]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '48 year', '58 year', '58 year of age',
       '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 20 1865', 'Aug 20th 1865',
       'Aug 31 1865', 'August 24 1865', 'August 29th 1865', 'July 22',
       'Maj 47', 'Maj 47th', 'Maj 47th PM', 'May 29th 1865', 'May 4',
       'May 47 PM', 'Sept 29 1865', 'Sept 29th 1865', 'Thirty nine',
       'twenty eight year'], dtype='<U26')

In [26]:
np.unique(DATE)

array(['1474', '16 day of October 1865', '16th day', '1865', '1865 1865',
       '1867', '20th day of August 1865', '22nd day of August 1865',
       '27th day', '28 day', '29th day', '29th day of September 1865',
       '3 day', '3 day of August 1865', '31 day', '31st day', '39 year',
       '6 day', '63 year', '7th day of September', 'Age 24 year',
       'August 24 1865', 'August 29th 1865', 'July 22', 'May 29th 1865',
       'May 4', 'May 47 PM', 'Miss 31st day', 'No1984', 'Sept 29th 1865'],
      dtype='<U26')

> Huge difference !!


In [27]:
len(P)

40

In [28]:
np.unique(Person)

array(['Amnesty Oath', 'Amnesty Oath Administered At', 'Andrew Johnson',
       'Anne H Elliott', 'Bray Lieut', 'Brown', 'Captn',
       'Captn Commanding', 'Characteristics', 'Charleston Sc',
       'City Miss', 'Copy Levi Steuben', 'Frank Conner', 'Frank E Miller',
       'Gibbs Henderson', 'Gry', 'Hazel', 'Hiram Smith', 'Hiserodt',
       'Hu Kennedy', 'JF Steinmeyer Sworn', 'Jacob Schroder Sworn',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Joseph Edings', 'Laborer', 'Pass Christian', 'Robert A',
       'Thomas Andrew Oath of', 'Triplicate AMNESTY', 'Walhalla',
       'Whaley', 'William Whaley Sworn', 'Woodville Wilkinson'],
      dtype='<U58')

In [29]:
P

['Anne H Elliott',
 'Andrew Johnson',
 'S Wilde Durson',
 'I E C Whaley',
 'E C Whaley',
 'Levi Steuben',
 'E',
 'ANDREW JOHNSON',
 'Levi Stuber Major',
 'Hu Kennedy',
 'JB Hiserodt',
 'Whaley',
 'William Whaley Sworn',
 'Levi Stuber',
 'I J F Steinmeyer',
 'JF Steinmeyer',
 'Joseph Edings',
 'Frank Geise',
 'Vicksburg',
 'Jacob Schroder',
 'Jacob Schroder Sworn',
 'JW Baxter',
 'J H Mathews',
 'Frank E Miller',
 'Bergevin',
 'P H Joor',
 'Frank Conner',
 'Caroline Belitzer',
 'caroline Belitzer',
 'Hiram Smith',
 'mark Sworn',
 'Geo D Carpenter',
 'Aiken Burnwell',
 'James E Thuring',
 'J E Thuring',
 'Robert A Wilson',
 'James D Kendall',
 'Holwell',
 'City',
 'JH Mathews']

**still ner-english-ontonotes-large works better than spacy**

****xlm-roberta-base-finetuned-panx-ner****
for Person

In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")
model = AutoModelForTokenClassification.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")

nlp1 = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/964 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

In [31]:
T7=[]
example = text

for i in range(20):
        ner_results = nlp1(example[i])
        T7.append(ner_results)

In [32]:
T7[1][1]

{'entity_group': 'ORG',
 'score': 0.86250067,
 'word': 'AmnestyProclamation',
 'start': 45,
 'end': 65}

In [33]:
Per2=[]
ORg2=[]
GPe2=[]
for i in range(20):
    for j in range(len(T7[i])):
        if ( T7[i][j]["entity_group"]=='ORG') & ((T7[i][j]["score"]) > 0.90):
            ORg2.append(T7[i][j]["word"])
        elif ( T7[i][j]["entity_group"]=='PER')  & ((T7[i][j]["score"]) > 0.85):
            Per2.append(T7[i][j]["word"])
        elif ( T7[i][j]["entity_group"]=='LOC') & ((T7[i][j]["score"]) > 0.84) :
            GPe2.append(T7[i][j]["word"])
GPe2=np.array(GPe2)
ORg2=np.array(ORg2)
Per2=np.array(Per2)



In [34]:
np.unique(Per2)

array(['ANDREWJOHNSON', 'AndrewJohnson', 'AnneHElliott',
       'CarolineBelitzer', 'FrankConner', 'HiramSmith', 'JFSteinmeyer',
       'JacobSchroder', 'LeviSteuben', 'RobertAWilson', 'WildeDurson'],
      dtype='<U16')

In [35]:
np.unique(Person)

array(['Amnesty Oath', 'Amnesty Oath Administered At', 'Andrew Johnson',
       'Anne H Elliott', 'Bray Lieut', 'Brown', 'Captn',
       'Captn Commanding', 'Characteristics', 'Charleston Sc',
       'City Miss', 'Copy Levi Steuben', 'Frank Conner', 'Frank E Miller',
       'Gibbs Henderson', 'Gry', 'Hazel', 'Hiram Smith', 'Hiserodt',
       'Hu Kennedy', 'JF Steinmeyer Sworn', 'Jacob Schroder Sworn',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Joseph Edings', 'Laborer', 'Pass Christian', 'Robert A',
       'Thomas Andrew Oath of', 'Triplicate AMNESTY', 'Walhalla',
       'Whaley', 'William Whaley Sworn', 'Woodville Wilkinson'],
      dtype='<U58')

In [36]:
np.unique(ORg2)

array(['AikenSC', 'AlmightyGod', 'AmnestyOath', 'AmnestyProclamation',
       'CharlestonSC',
       'ConstitutionofUnitedStatesUnionofStatesthereunder', 'IWmWhaley',
       'LawsProclamationsmadeexistingrebellionreferenceemancipationofslave',
       'Oath', 'OrangeburgSC',
       'UNITEDSTATESFORCESNo1282PROVOSTMARSHALSOFFICE', 'VicksburgMiss',
       'WAFleitasAmnestyOath'], dtype='<U66')

****

# best result until now

***Person***

In [37]:
np.unique(Per2)

array(['ANDREWJOHNSON', 'AndrewJohnson', 'AnneHElliott',
       'CarolineBelitzer', 'FrankConner', 'HiramSmith', 'JFSteinmeyer',
       'JacobSchroder', 'LeviSteuben', 'RobertAWilson', 'WildeDurson'],
      dtype='<U16')

In [38]:
np.unique(P)

array(['ANDREW JOHNSON', 'Aiken Burnwell', 'Andrew Johnson',
       'Anne H Elliott', 'Bergevin', 'Caroline Belitzer', 'City', 'E',
       'E C Whaley', 'Frank Conner', 'Frank E Miller', 'Frank Geise',
       'Geo D Carpenter', 'Hiram Smith', 'Holwell', 'Hu Kennedy',
       'I E C Whaley', 'I J F Steinmeyer', 'J E Thuring', 'J H Mathews',
       'JB Hiserodt', 'JF Steinmeyer', 'JH Mathews', 'JW Baxter',
       'Jacob Schroder', 'Jacob Schroder Sworn', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Levi Steuben', 'Levi Stuber',
       'Levi Stuber Major', 'P H Joor', 'Robert A Wilson',
       'S Wilde Durson', 'Vicksburg', 'Whaley', 'William Whaley Sworn',
       'caroline Belitzer', 'mark Sworn'], dtype='<U20')

****

***Organization***

In [39]:
np.unique(ORg2)

array(['AikenSC', 'AlmightyGod', 'AmnestyOath', 'AmnestyProclamation',
       'CharlestonSC',
       'ConstitutionofUnitedStatesUnionofStatesthereunder', 'IWmWhaley',
       'LawsProclamationsmadeexistingrebellionreferenceemancipationofslave',
       'Oath', 'OrangeburgSC',
       'UNITEDSTATESFORCESNo1282PROVOSTMARSHALSOFFICE', 'VicksburgMiss',
       'WAFleitasAmnestyOath'], dtype='<U66')

In [40]:
np.unique(OR)

array(['Amnesty', 'Harrison Co', 'UNITED STATES FORCES', 'USCI',
       'Union of States', 'Walhalla'], dtype='<U20')

In [41]:
np.unique(GP)

array(['Charleston', 'Charleston S C', 'Charleston SC',
       'Mississippi City', 'Orangeburg', 'So Carolina', 'South Carolina',
       'UNITED STATES OF', 'US', 'United State of America',
       'United States', 'United States of America'], dtype='<U24')

****

***GPE and location***

In [42]:
np.unique(GPE)

array(['Charleston', 'Gainsville', 'Mississippi', 'Mississippi City',
       'NC', 'New Orleans', 'South Carolina', 'States', 'US',
       'United States', 'United States of America', 'Vicksburg'],
      dtype='<U24')

****

***Date***

In [43]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '48 year', '58 year', '58 year of age',
       '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 20 1865', 'Aug 20th 1865',
       'Aug 31 1865', 'August 24 1865', 'August 29th 1865', 'July 22',
       'Maj 47', 'Maj 47th', 'Maj 47th PM', 'May 29th 1865', 'May 4',
       'May 47 PM', 'Sept 29 1865', 'Sept 29th 1865', 'Thirty nine',
       'twenty eight year'], dtype='<U26')

****

****

**pattern**

* num +... +Capital_letter+... + num
* Cap_letter + num + num

In [44]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '48 year', '58 year', '58 year of age',
       '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 20 1865', 'Aug 20th 1865',
       'Aug 31 1865', 'August 24 1865', 'August 29th 1865', 'July 22',
       'Maj 47', 'Maj 47th', 'Maj 47th PM', 'May 29th 1865', 'May 4',
       'May 47 PM', 'Sept 29 1865', 'Sept 29th 1865', 'Thirty nine',
       'twenty eight year'], dtype='<U26')

In [45]:
len(Date)

43

In [46]:
import regex as re
Pattern1=re.compile(r'^\d\.*.*\d\d$')
Pattern2=re.compile(r'^[A-Z]\.*.*\d \.*.*\d\d$')

In [47]:
M=[]
for i in Date:
        date=re.search(r'^\d\.*.*\d\d$' ,i)
        date2=re.search(r'^[A-Z]\.*.*\d \.*.*\d\d$' ,i)
        M.append(date)
        M.append(date2)
        

In [48]:
F=[]
for i in M:
    if i!=None:
        F.append(i.group())

In [49]:
F

['27th day of Novb 1865',
 '27th day of Novbr 1865',
 '28 day of Sept 1865',
 '31 day of Oct 1865',
 '31st day of August AD 1865',
 'August 24 1865',
 '3 day of August 1865',
 '3 day of Augt 1865',
 '16th day of Sept 1865',
 '16 day of October 1865',
 '6 day of Nov 1865',
 'Sept 29 1865',
 '29th day of September 1865',
 '29th day of Sept 29 1865',
 '3d day of February 1868',
 '7th day of September 186',
 'Aug 20 1865',
 '20th day of August 1865',
 '22nd day of August 1865',
 '18th Sept 1865',
 'Aug 31 1865',
 '31st day of Aug 1865',
 '31 day of Aug 1865']

> great!

In [50]:
P

['Anne H Elliott',
 'Andrew Johnson',
 'S Wilde Durson',
 'I E C Whaley',
 'E C Whaley',
 'Levi Steuben',
 'E',
 'ANDREW JOHNSON',
 'Levi Stuber Major',
 'Hu Kennedy',
 'JB Hiserodt',
 'Whaley',
 'William Whaley Sworn',
 'Levi Stuber',
 'I J F Steinmeyer',
 'JF Steinmeyer',
 'Joseph Edings',
 'Frank Geise',
 'Vicksburg',
 'Jacob Schroder',
 'Jacob Schroder Sworn',
 'JW Baxter',
 'J H Mathews',
 'Frank E Miller',
 'Bergevin',
 'P H Joor',
 'Frank Conner',
 'Caroline Belitzer',
 'caroline Belitzer',
 'Hiram Smith',
 'mark Sworn',
 'Geo D Carpenter',
 'Aiken Burnwell',
 'James E Thuring',
 'J E Thuring',
 'Robert A Wilson',
 'James D Kendall',
 'Holwell',
 'City',
 'JH Mathews']